## Expense Tracker

In this lab we'll bring together what we've learned about structured data and function calling to create an application that helps us track our expenses. We'd like to take some user input and allow an agent to decide which tools are required to complete the users request.

Again, refer to the [function calling guide](https://platform.openai.com/docs/guides/function-calling) for help.

## Step 1: Setup

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

MODEL = 'gpt-4o-mini'

## Step 2: Defining Application Logic

Before we define an agent, we need something for it to interact with. We’ll build a simple expense tracker that supports three core actions:

- Add expenses (e.g., from a memo like “I spent $20 on groceries”)
- View a summary of expenses (totals, counts, breakdowns)
- List all expenses

> 💡 In yesterday’s lab, our `Expense` objects included a date attribute. For now, ignore dates to keep the exercise focused. You’ll have a chance to tackle date support in the bonus section later if you want to.

### Part A: Design the Expense Data Structure

Decide how to represent expenses in your program. For example, you might use a list of dictionaries, where each dictionary has fields like `amount`, `category`, and `description`.

### Part B: Implement Application Functions

Write Python functions that implement the three actions:

- `add_expense`: Add one or more expenses to your data structure.
  Tip: You can reuse the `extract_expenses_data` function from yesterday's lab if you like, but this is optional. Function calling can also handle some data extraction for you, so choose whichever approach feels most intuitive. Returns a list of objects representing the expenses that were added.
- `summarize_expenses`: Returns a summary object (total spent, number of expenses, breakdown by category).
- `list_expenses`: Return the list of all recorded expenses.

> 💡 The 'Example Usage' is just for demonstration. You can design your objects however you'd like.

In [ ]:
class ExpenseTracker:

    # TODO

    pass


# Example Usage:
my_tracker = ExpenseTracker()

print(my_tracker.add_expense(100.0, "Groceries", "Groceries"))
# amount=100.0 category='Groceries' description='Groceries'

print(my_tracker.add_expense(25.0, "Eating Out", "Lunch with friends"))
# amount=25.0 category='Eating Out' description='Lunch with friends'

print(my_tracker.summarize_expenses())
# {
#     "total_spent": 125.00,
#     "number_of_expenses": 2,
#     "breakdown": {
#         "Groceries": 100.00,
#         "Eating Out": 25.00
#     }
# }

print(my_tracker.list_expenses())
# [Expense(amount=100.0, category='Groceries', description='Groceries'), Expense(amount=25.0, category='Eating Out', description='Lunch with friends')]

## Step 3: Getting Agentic

It's time to build our agent. You'll need:

- **Tool descriptions.** Make sure the descriptions are clear, match your function signatures, and align with the expected shape.
- **An agent.** Your agent should take a user query as input and use it's available tools to decide what to do.

> 💡 Depending on your implementation, you might have trouble with the 'category' of the 'Lunch with friends' expense. We'll look at this problem in the bonus exercises.

For now, we can use logging to see that our tools are being called correctly. In the next step, we'll add a user interface to share what's happening behind the scenes.

In [ ]:
# TODO: Define tools and agent to use them

# Example Usage:

my_tracker = ExpenseTracker()

print(handle_expense_query(my_tracker, "I spent $100 on groceries yesterday"))
# [Expense(amount=100.0, category='Groceries', description='Groceries')]

print('--------------------------------')

print(handle_expense_query(my_tracker, "twenty-five dollars on lunch with friends"))
# [Expense(amount=25.0, category='Eating Out', description='Lunch with friends')]

print('--------------------------------')

print(handle_expense_query(my_tracker, "Show me a summary of my expenses"))
# {
#     "total_spent": 125.00,
#     "number_of_expenses": 2,
#     "breakdown": {
#         "Groceries": 100.00,
#         "Eating Out": 25.00
#     }
# }

print('--------------------------------')

print(handle_expense_query(my_tracker, "List all my expenses"))
# [Expense(amount=100.0, category='Groceries', description='Groceries'), Expense(amount=25.0, category='Eating Out', description='Lunch with friends')]

## Step 4: Using our Tool Call Results

Now that we can see our tools are being called correctly, we need to do something with their results. Some of our tools have side effects, and some of them just fetch information for us. Our next step is to bundle up all of our context and make one final LLM call to get a response for our user.

Instead of something like `[Expense(amount=25.0, category='Eating Out', description='Lunch with friends')]` to indicate that we've added a new expense, our user should see something like:

> Great! I've added a new expense of $25 with the category of "Eating Out" and a description of "Lunch with friends." Let me know if you have any other expenses you'd like to add.

In [ ]:
# TODO: Update `handle_expense_query` to return a user-friendly message about what it's
#       accomplished rather than logging the results of tool calls.

# Example Usage:

my_tracker = ExpenseTracker()

print(handle_expense_query(my_tracker, "I spent $100 on groceries yesterday"))
# Great job on keeping track of your expenses! I've noted that you spent $100 on groceries yesterday. If you need any more help with your budgeting or tracking other expenses, just let me know!

print('--------------------------------')

print(handle_expense_query(my_tracker, "twenty-five dollars on lunch with friends"))
# Perfect! I've added a new expense of $25 with the category of "Eating Out" and a description of "Lunch with friends." Would you like to add another expense?

print('--------------------------------')

print(handle_expense_query(my_tracker, "Show me a summary of my expenses"))
# Here's a summary of your expenses:
# Total spent: $125.00
# Number of expenses: 2
# Breakdown:
#   Groceries: $100.00
#   Eating Out: $25.00
#
# Let me know if you'd like to add another expense or see a list of all your expenses.

print('--------------------------------')

print(handle_expense_query(my_tracker, "List all my expenses"))
# I've gathered all your expenses for you! Here’s what I found:
#   1. **Groceries**: $100.00 (Grocery shopping yesterday)
#   2. **Eating Out**: $25.00 (Lunch with friends)
#
# Let me know if you'd like to add another expense or see a summary of your expenses.

## Step 5: Get User Input + Bonuses

Set up a simple interface (command line, Gradio, etc.) to test how your agent manages expenses.

### Bonus Challenges

Think about these questions as ways to stretch your app design. You don’t need to implement them, but consider the trade-offs and implications, and have a go if you have time.

1. **Multiple expenses in one input**
    - Example: “Yesterday I spent $100 on groceries and $20 on lunch.”
    - Should this create two expenses at once, or should the agent call add_expense twice?

2. **Category consistency**
    - How are categories assigned?
    - What happens if similar phrases create slightly different categories (e.g., “dining out” vs. “restaurant”)?
    - Should category logic live in the agent, in `add_expense`, or somewhere else?

3. **Multi-step queries**
    - Example: “Add an expense for eating out, $40, then show me my updated summary.”
    - How should the system handle a request that chains multiple actions?

4. **Category- or time-specific summaries**
    - Example: “How much have I spent on groceries this month?”
    - What extra logic is needed to support filtering by category and/or date?

5. **Searching or editing expenses**
    - Could we provide a way for users to search for expenses? How could that work?

In [ ]:
# TODO: Build Expense Tracker Interface